### Direct prediction of coupled quantiles

This distribution is constructed such that the quantiles are in ordered fashion, cannot be zero, and have smooth derivatives. The activation $\exp$ is chosen in favour of ReLU to always have smooth derivatives (this is zero for ReLU with $a<0$). By construction, the complete distribution scales with the median quantile, and all quantiles have the nonzero constraint by choosing activation functions sigmoid or $\exp$ for the parameters defined below. 

Similar to the model above, the center quantile is predicted directly (i.e. with a single parameter). The other parameters are scaled versions of this, based on the calculation below. The quantiles are abbreviated as: $q_m = q_{0.5}$, $q_0 = q_{0.005}$, $q_1 = q_{0.025}$, $q_2 = q_{0.165}$, $q_3 = q_{0.25}$, $q_5 = q_{0.75}$, $q_6 = q_{0.835}$, $q_7 = q_{0.975}$, $q_8 = q_{0.995}$.  

The median:
$$
q_m = q_m, \text{with } q_m \in [0, \infty]\text{, by } q_m=\exp(a_4)
$$

The extremes, $q_0$ and $q_8$:  
$$
\begin{align}
q_0 &= \alpha \cdot q_m, &\text{with } \alpha \in [0, 1]\text{, by } \alpha=\text{sigmoid}(a_0)\\
q_8 &= (1 + \delta) \cdot q_m, &\text{with } \delta \in [0, \infty)\text{, by } \delta=\exp(a_9)\\
\end{align}
$$

The intermediate values are between these, with the relative posistion in the range $[0, 1]$:  
$$
\begin{align}
q_2 &= q_0 + \beta \cdot (q_m - q_0), &\text{with } \beta \in [0, 1]\text{, by } \beta=\text{sigmoid}(a_2)\\
    &= (\alpha + \beta - \alpha\cdot\beta)q_m\\
q_1 &= q_0 + \gamma (q_2 - q_0), &\text{with } \gamma \in [0, 1]\text{, by } \gamma=\text{sigmoid}(a_1)\\
    &= (\alpha + \gamma \beta - \alpha \beta \gamma)q_m
\end{align}
$$

The same method is applied for $q_3, q_5, q_6, q_7$, resulting in:
$$
\begin{align}
q_3 &= q_2 + \epsilon \cdot (q_m - q_2) &= (\alpha + \beta - \alpha \beta + \epsilon - [\alpha + \beta - \alpha\beta]\epsilon)q_m \\
q_6 &= q_m + \zeta (q_8-q_m) &= (1+\zeta \delta)q_m\\
q_5 &= q_m + \eta (q_6-q_m) &= (1+\eta \zeta \delta) q_m\\
q_7 &= q_6 + \theta (q_8-q_6) &= (1+\zeta \delta + \theta \delta+\theta\eta\zeta\delta)q_m
\end{align}
$$

In which $\epsilon, \zeta, \eta, \theta$ are in the interval $[0, 1]$ by setting the sigmoid as activation function. 

In [29]:
def get_custom_layer(i):
    def custom_layer(tensor):
        # unpack input
        qm = tensor[0]

        # parameters
        alpha   = tensor[1]
        beta    = tensor[2]
        gamma   = tensor[3]
        delta   = tensor[4]
        epsilon = tensor[5]
        zeta    = tensor[6]
        eta     = tensor[7]
        theta   = tensor[8]
        
        if i == 0:
            return alpha * qm
        elif i == 1:
            return (alpha + gamma*beta - alpha*beta*gamma) * qm
        elif i == 2:
            return (alpha + beta - alpha * beta) * qm
        elif i == 3:
            return (alpha + beta - alpha*beta + epsilon - (alpha + beta - alpha*beta)*epsilon) * qm
        elif i == 5:
            return (1 + eta*zeta*delta) * qm
        elif i == 6:
            return (1 + zeta*delta) * qm
        elif i == 7:
            return (1 + zeta*delta + theta*delta + theta*eta*zeta*delta)*qm
        elif i == 8:
            return (1 + delta) * qm
        
    return custom_layer


def get_direct_dist_model(inp_shape, num_nodes=256):
    # clear previous sessions
    K.clear_session()

    inp = Input(inp_shape, name="input")
    x = inp
    x = Dense(num_nodes, activation='relu')(x)
    x = Dense(num_nodes, activation='relu')(x)
    x = Dense(num_nodes, activation='relu')(x)
    
    # direct prediction of median
    qm = Dense(1, name="q4", activation="exponential")(x)
    
    # setup parameters
    alpha   = Dense(1, name="alpha",   activation="sigmoid")(x)
    beta    = Dense(1, name="beta",    activation="sigmoid")(x)
    gamma   = Dense(1, name="gamma",   activation="sigmoid")(x)
    delta   = Dense(1, name="delta",   activation="exponential")(x)
    epsilon = Dense(1, name="epsilon", activation="sigmoid")(x)
    zeta    = Dense(1, name="zeta",    activation="sigmoid")(x)
    eta     = Dense(1, name="eta",     activation="sigmoid")(x)
    theta   = Dense(1, name="theta",   activation="sigmoid")(x)
    
    
    outs = []

    for i in range(9):
        if i == 4:
            out_q = qm
        else:
            custom_layer = get_custom_layer(i)
            params = [qm, alpha, beta, gamma, delta, epsilon, zeta, eta, theta]
            out_q = Lambda(custom_layer, name="q{}".format(i))(params)
        outs.append(out_q)
    
    model = Model(inputs=inp, outputs=outs) 

    return model